# COVID-19 Data

Tracking COVID-19 using the Johns Hopkins CSSE data hosted in [GitHub](https://github.com/CSSEGISandData/COVID-19)

## Load the Data

Load the CSV data from GitHub using Pandas

In [1]:
import pandas as pd

In [2]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

In [3]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,990,990,990,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,410,411,413,414,414,418,418,422,426,428
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,296,296,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,102,119,120,124


## Filter Down to Canada

In [4]:
def filter_by_country(df, country):
    is_country = df['Country/Region']==country
    return df[is_country]

In [5]:
canada_confirmed = filter_by_country(confirmed, 'Canada')
canada_deaths = filter_by_country(deaths, 'Canada')
canada_recovered = filter_by_country(recovered, 'Canada')

In [6]:
canada_confirmed

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
44,"Toronto, ON",Canada,43.6532,-79.3832,0,0,0,0,1,1,...,5,10,14,17,19,19,21,24,27,28
45,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,7,8,8,8,9,12,13,21,21,27
57,"London, ON",Canada,42.9849,-81.2453,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
102,"Montreal, QC",Canada,45.5017,-73.5673,0,0,0,0,0,0,...,1,1,1,1,1,1,2,2,3,4
174,"Calgary, Alberta",Canada,51.0447,-114.0719,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
207,"Edmonton, Alberta",Canada,53.5461,-113.4938,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3


## The last column is the most recent data

In [7]:
canada_confirmed.iloc[:,-1]

44     28
45     27
57      1
102     4
174     1
207     3
Name: 3/8/20, dtype: int64

In [8]:
canada_confirmed.T.iloc[[0,-1]]

,44,45,57,102,174,207
Province/State,"Toronto, ON",British Columbia,"London, ON","Montreal, QC","Calgary, Alberta","Edmonton, Alberta"
3/8/20,28,27,1,4,1,3


In [19]:
top = canada_confirmed.T.head(4)
bottom = canada_confirmed.T.tail(1)
canada_confirmed_latest = pd.concat([top, bottom]).T
canada_confirmed_latest

,Province/State,Country/Region,Lat,Long,3/8/20
44,"Toronto, ON",Canada,43.6532,-79.3832,28
45,British Columbia,Canada,49.2827,-123.121,27
57,"London, ON",Canada,42.9849,-81.2453,1
102,"Montreal, QC",Canada,45.5017,-73.5673,4
174,"Calgary, Alberta",Canada,51.0447,-114.072,1
207,"Edmonton, Alberta",Canada,53.5461,-113.494,3


## Map the latest numbers

Ensure that the `ipyleaflet` and `geopandas` modules are installed with Pip or Conda and make sure the Jupyter extension is also installed.

### Using pip

```
pip install ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet  # can be skipped for notebook 5.3 and above
```

### Using conda

```
conda install -c conda-forge ipyleaflet
```

### JupyterLab extension

If you have JupyterLab, you will also need to install the JupyterLab extension:

```
jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
```

In [23]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker
import geopandas

Convert the Pandas `DataFrame` to a `GeoDataFrame` so we can map it.

In [27]:
geometry = geopandas.points_from_xy(canada_confirmed_latest.Long, canada_confirmed_latest.Lat)
gdf = geopandas.GeoDataFrame(canada_confirmed_latest.iloc[:,-1], geometry=geometry)
canada_confirmed_latest.iloc[:,-1]

44      POINT (-79.38320 43.65320)
45     POINT (-123.12070 49.28270)
57      POINT (-81.24530 42.98490)
102     POINT (-73.56730 45.50170)
174    POINT (-114.07190 51.04470)
207    POINT (-113.49380 53.54610)
Name: geometry, dtype: geometry

In [14]:
center = (50.5990364,-93.3975467)
watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)
m = Map(layers=(watercolor, ), center=center, zoom=4)
display(m)

Map(center=[50.5990364, -93.3975467], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [18]:
for row in canada_confirmed_latest.T:
    display(row)
    
canada_confirmed_latest.T

'Province/State'

'Country/Region'

'Lat'

'Long'

'3/8/20'

,Province/State,Country/Region,Lat,Long,3/8/20
44,"Toronto, ON",Canada,43.6532,-79.3832,28
45,British Columbia,Canada,49.2827,-123.121,27
57,"London, ON",Canada,42.9849,-81.2453,1
102,"Montreal, QC",Canada,45.5017,-73.5673,4
174,"Calgary, Alberta",Canada,51.0447,-114.072,1
207,"Edmonton, Alberta",Canada,53.5461,-113.494,3
